In [1]:
import requests
from bs4 import BeautifulSoup
from itertools import zip_longest
import pandas as pd
from tqdm import tqdm

In [2]:
base_url = 'https://banhae.pet/'
brand_list = 'suv/brand_list.php'
url = base_url+brand_list+'?page=1&pageNum=20&co_page=1&co_fage=1#tab2'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

pagination = soup.select('.pagination.pagination-sm a[href]')

href_list = [base_url+brand_list+page.get('href') for page in pagination if page]
href_list.insert(0, url)

print(href_list)

['https://banhae.pet/suv/brand_list.php?page=1&pageNum=20&co_page=1&co_fage=1#tab2', 'https://banhae.pet/suv/brand_list.php?page=2&pageNum=20&co_page=2&co_fage=2#tab2', 'https://banhae.pet/suv/brand_list.php?page=3&pageNum=20&co_page=3&co_fage=3#tab2', 'https://banhae.pet/suv/brand_list.php?page=4&pageNum=20&co_page=4&co_fage=4#tab2', 'https://banhae.pet/suv/brand_list.php?page=5&pageNum=20&co_page=5&co_fage=5#tab2', 'https://banhae.pet/suv/brand_list.php?page=6&pageNum=20&co_page=6&co_fage=6#tab2']


In [3]:
brand_pages = []

for href in tqdm(href_list):
    response = requests.get(href)
    soup = BeautifulSoup(response.content, 'html.parser')

    links = soup.select('.brand_lisr a[href]')

    for link in links:
        brand_pages.append(base_url+link['href'])

print(brand_pages)

100%|██████████| 6/6 [00:05<00:00,  1.14it/s]

['https://banhae.pet//goods/goods_list.php?brandCd=052&sno=1', 'https://banhae.pet//goods/goods_list.php?brandCd=006&sno=124', 'https://banhae.pet//goods/goods_list.php?brandCd=059&sno=19', 'https://banhae.pet//goods/goods_list.php?brandCd=055&sno=9', 'https://banhae.pet//goods/goods_list.php?brandCd=049&sno=164', 'https://banhae.pet//goods/goods_list.php?brandCd=005&sno=5', 'https://banhae.pet//goods/goods_list.php?brandCd=116&sno=186', 'https://banhae.pet//goods/goods_list.php?brandCd=094&sno=154', 'https://banhae.pet//goods/goods_list.php?brandCd=004&sno=8', 'https://banhae.pet//goods/goods_list.php?brandCd=091&sno=141', 'https://banhae.pet//goods/goods_list.php?brandCd=122&sno=193', 'https://banhae.pet//goods/goods_list.php?brandCd=008&sno=183', 'https://banhae.pet//goods/goods_list.php?brandCd=009&sno=170', 'https://banhae.pet//goods/goods_list.php?brandCd=057&sno=14', 'https://banhae.pet//goods/goods_list.php?brandCd=056&sno=12', 'https://banhae.pet//goods/goods_list.php?brandCd=

In [4]:
brand_info_pages = []

for brand_page in tqdm(brand_pages):
    response = requests.get(brand_page)
    soup = BeautifulSoup(response.content, 'html.parser')

    pagination = soup.select_one('.pagination.pagination-sm')
    pagination = pagination.select('a[href]')

    href_list = [brand_page[:40]+page.get('href') for page in pagination if page]
    href_list.insert(0, brand_page)

    brand_info_pages.append(href_list)

print(brand_info_pages)

100%|██████████| 102/102 [01:39<00:00,  1.03it/s]

[['https://banhae.pet//goods/goods_list.php?brandCd=052&sno=1', 'https://banhae.pet//goods/goods_list.php?page=2&sno=1&pageNum=6&sort=review&co_page=2&co_fage=2#tab2', 'https://banhae.pet//goods/goods_list.php?page=3&sno=1&pageNum=6&sort=review&co_page=3&co_fage=3#tab2', 'https://banhae.pet//goods/goods_list.php?page=4&sno=1&pageNum=6&sort=review&co_page=4&co_fage=4#tab2', 'https://banhae.pet//goods/goods_list.php?page=5&sno=1&pageNum=6&sort=review&co_page=5&co_fage=5#tab2', 'https://banhae.pet//goods/goods_list.php?page=6&sno=1&pageNum=6&sort=review&co_page=6&co_fage=6#tab2'], ['https://banhae.pet//goods/goods_list.php?brandCd=006&sno=124'], ['https://banhae.pet//goods/goods_list.php?brandCd=059&sno=19'], ['https://banhae.pet//goods/goods_list.php?brandCd=055&sno=9', 'https://banhae.pet//goods/goods_list.php?page=2&sno=9&pageNum=6&sort=review&co_page=2&co_fage=2#tab2', 'https://banhae.pet//goods/goods_list.php?page=3&sno=9&pageNum=6&sort=review&co_page=3&co_fage=3#tab2', 'https://banh

In [5]:
product_numbers = []

for _ in tqdm(brand_info_pages):
    for brand_info_page in _:
        try:
            response = requests.get(brand_info_page)
            soup = BeautifulSoup(response.content, 'html.parser')

            thumbnails = soup.select_one('.item-display.type-gallery.food_list')
            thumbnails = thumbnails.select('.txt a[href]')

            product_number = [link.get('href')[25:] for link in thumbnails]
            product_numbers.extend(product_number)

        except:
            print(f'There are no products on the {brand_info_page}')
            pass

print(product_numbers)

 62%|██████▏   | 63/102 [01:41<00:43,  1.13s/it]

There are no products on the https://banhae.pet//goods/goods_list.php./goods_list.php?page=2&brandCd=002&sno=95


 75%|███████▍  | 76/102 [02:03<00:41,  1.61s/it]

There are no products on the https://banhae.pet//goods/goods_list.php./goods_list.php?page=2&brandCd=010&sno=115


100%|██████████| 102/102 [02:34<00:00,  1.52s/it]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '38', '39', '40', '41', '42', '43', '44', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '34', '35', '37', '609', '694', '695', '696', '697', '698', '699', '45', '46', '47', '108', '109', '110', '628', '629', '630', '111', '112', '113', '114', '420', '421', '422', '423', '424', '425', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '134', '133', '132', '610', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '15

In [6]:
df = pd.DataFrame()

for product_number in tqdm(product_numbers):
    detail = f'https://banhae.pet/suv/food_detail.php?sno={product_number}'

    response = requests.get(detail)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_df = pd.DataFrame({'상품명':[soup.select_one('.food-title').text]})

    info = soup.select_one('.info-wrap')

    info_p = info.select('ul li p')
    info_span = info.select('ul li span')

    info_df = pd.DataFrame({s.text:[p.text] for p, s in zip_longest(info_p, info_span, fillvalue='')})

    ingredient_info = soup.select('.ingredient-wrap .info')
    ingredient_result = soup.select('.ingredient-wrap .result')

    ingredient_df = pd.DataFrame({i.text:[r.text] for i, r in zip(ingredient_info, ingredient_result)})

    material = f'https://banhae.pet/suv/food_material.php?sno={product_number}'

    response = requests.get(material)
    soup = BeautifulSoup(response.content, 'html.parser')

    material = soup.select('.material_list ul span')

    material_df = pd.DataFrame({'주성분':[[tag.text.strip() for tag in material]]})

    df = pd.concat([df, pd.concat([product_df, info_df, ingredient_df, material_df], axis=1)])

df

100%|██████████| 676/676 [17:09<00:00,  1.52s/it]


,상품명,권장 연령,권장 사이즈,사료 종류,사료 형태,주 단백질원,알갱이 크기,제조국,원산지,조단백,조지방,조섬유,조회분,수분,칼슘,인,오메가3,오메가6,기타 영양소,주성분
0,ANF 홀리스틱 그레인프리 Dog 양고기 & 감자,전연령,ALL,주식,건식,,,호주,호주,28%,15%,5%,10%,10%,1.45%,1.5%,0%,0%,,"[양고기, 감자, 완두콩, 병아리콩, 고구마, 정제계유, 렌틸콩, 비트펄프, 건조계..."
0,ANF 홀리스틱 그레인프리 Dog 닭고기 & 감자,전연령,ALL,주식,건식,,,호주,호주,28%,15%,5%,10%,10%,1.45%,1.5%,0%,0%,,"[닭고기, 감자, 완두콩, 병아리콩, 고구마, 정제계유, 렌틸콩, 비트펄프, 연어,..."
0,ANF 홀리스틱 그레인프리 Dog 연어 & 병아리콩,전연령,ALL,주식,건식,,,호주,호주,26%,15%,3.5%,8.5%,10%,1%,0.9%,0%,0%,대사에너지 (3526 Kcal/kg),"[연어, 병아리콩, 노란콩, 노란콩전분, 정제계유, 감귤섬유소, 토마토박, 아마씨,..."
0,ANF 홀리스틱,전연령,ALL,주식,건식,,,호주,호주,24%,15%,3.5%,8.5%,0%,1.2%,1.5%,0%,0%,,"[닭고기, 현미, 닭기름(토코페롤 및 팔미트산 아스코르빌의 혼합물로 보존), 비트펄..."
0,ANF 램 홀리스틱,전연령,ALL,주식,건식,,,호주,호주,22%,12%,4.5%,11%,0%,1.45%,1.2%,0%,0%,,"[양고기, 현미, 귀리, 쌀겨, 아마씨, 닭기름(토코페롤 및 팔미트산 아스코르빌의 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,허즈 미국산 오리가슴살,ALL,ALL,주식용,에어드라이,순살 오리,확인불가,대만,대만,28%,18%,2%,3%,14%,0.5%,0.4%,0%,0%,,"[오리가슴살, 분리단백(완두콩), 코코넛글리세린, 호박가루, 들깨가루, 호박씨가루,..."
0,헤일로 닭고기와 닭간 퍼피,퍼피,ALL,주식,건식,,0.9Cm,미국,미국,28%,18%,5%,9%,10%,1.2%,1%,0.8%,3.75%,,"[전란, 닭고기, 닭간, 귀리, 보리, 완두콩분말, 콩단백질, 완두콩단백질, 계유,..."
0,헤일로 닭고기와 닭간 스몰브리드,전연령,소형견,주식,건식,,,미국,미국,27%,17%,5%,9%,10%,0.5%,0.4%,0.9%,3.75%,,"[닭고기, 닭간, 전란, 귀리, 완두콩분말, 보리, 콩단백질, 계유, 아마씨, 완두..."
0,헤일로 연어와 흰살생선 스몰브리드,전연령,소형견,주식,건식,,,미국,미국,27%,12%,5%,9%,10%,0.5%,0.4%,1.1%,2.4%,,"[생선, 연어, 렌틸콩, 전란, 완두콩분말, 건조병아리콩, 콩단백질, 고구마분말, ..."


In [8]:
df.to_csv('product_details.csv', index=False, encoding='utf-8-sig')